In [68]:
from __future__ import print_function
import mackinac
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.core.model import copy
import os
from os.path import join
from cobra.flux_analysis.parsimonious import add_pfba
import sys
from cobra.util.solver import linear_reaction_coefficients
from optlang.interface import OPTIMAL
from itertools import chain
from optlang.symbolics import Zero
from cobra.util import solver as sutil
from cobra.core.solution import get_solution

In [3]:
# Create a profile on PATRIC https://www.patricbrc.org/
# I highly suggest you use PATRIC rather than ModelSEED directly, it allows for gathering more 
# information about your microbe of interest. You'll probably want to start with an array of
# different strains all from the same species you are interested in. We can talk more about this
# idea during a call. 
username = ''
mackinac.get_token(username) # Use your PATRIC database username here

patric password: ········


u'tjmoutinho@patricbrc.org'

In [ ]:
## Only run this once to limit any confusion later on.
## Check to make sure it completes on your PATRIC profile: www.patricbrc.org/job/
## If it failes to complete, just run it again until it completes. 

mackinac.create_patric_model(genome_id,genome_id)

In [4]:
genome_id = '266.10'

model = mackinac.create_cobra_model_from_patric_model(genome_id)

Script directory:  /home/tjmoutinho/Lactobacillus/Code


In [42]:
# I wouldn't suggest you worry about the charge imbalanced reactions for right now.
# The code you sent had a useless 'validation' section that didn't remove the exchange reactions
# before checking for imbalances. All EX_ (exchange), SK_ (sink), and DM_ (demand) reactions are
# by nature imbalanced, this is how it should be. 

imbalanced_reactions = []
for rxn in model.reactions:
    if not rxn.id.startswith('EX') | rxn.id.startswith('bio1') | rxn.id.startswith('SK'):
        rxn_mass_dict = rxn.check_mass_balance()
        if bool(rxn_mass_dict) == True:
            imbalanced_reactions.append([rxn,rxn_mass_dict])
imbalanced_reactions

[[<Reaction rxn05404_c at 0x7f09217de050>, {'charge': 1.0}],
 [<Reaction rxn05444_c at 0x7f09217de610>, {'charge': -1.0}],
 [<Reaction rxn05459_c at 0x7f092178f650>, {'charge': -1.0}],
 [<Reaction rxn05453_c at 0x7f092178f890>, {'charge': -1.0}],
 [<Reaction rxn05350_c at 0x7f092175f110>, {'charge': -1.0}],
 [<Reaction rxn05358_c at 0x7f092175f710>, {'charge': 1.0}],
 [<Reaction rxn05437_c at 0x7f092175fe50>, {'charge': -1.0}],
 [<Reaction rxn05408_c at 0x7f09217703d0>, {'charge': 1.0}],
 [<Reaction rxn05324_c at 0x7f0921700090>, {'charge': -1.0}],
 [<Reaction rxn05458_c at 0x7f09216f6850>, {'charge': -1.0}],
 [<Reaction rxn05450_c at 0x7f0921687250>, {'charge': -1.0}],
 [<Reaction rxn05443_c at 0x7f09216877d0>, {'charge': -1.0}],
 [<Reaction rxn05359_c at 0x7f0921662950>, {'charge': 1.0}],
 [<Reaction rxn05327_c at 0x7f0921604290>, {'charge': -1.0}],
 [<Reaction rxn05322_c at 0x7f09215a0cd0>, {'charge': -2.0}],
 [<Reaction rxn05429_c at 0x7f092151a150>, {'charge': 1.0}],
 [<Reaction r

In [5]:
# List of Genomic Features from PATRIC
feature_list = mackinac.get_genome_features(genome_id, annotation='PATRIC')

In [6]:
feature_list

[{u'aa_length': 552,
  u'aa_sequence_md5': u'c8141f44119381d979a1acf3eba7a7b9',
  u'accession': u'FNEA01000020',
  u'annotation': u'PATRIC',
  u'date_inserted': u'2016-11-19T18:47:57.167Z',
  u'date_modified': u'2016-11-19T18:47:57.167Z',
  u'end': 33752,
  u'feature_id': u'PATRIC.266.10.FNEA01000020.CDS.32094.33752.rev',
  u'feature_type': u'CDS',
  u'figfam_id': u'FIG00094176',
  u'genome_id': u'266.10',
  u'genome_name': u'Paracoccus denitrificans strain DSM 413',
  u'go': [u'GO:0016153|urocanate hydratase activity'],
  u'location': u'complement(32094..33752)',
  u'na_length': 1659,
  u'na_sequence_md5': u'204df824ca7c3aaa88baa15636008d1f',
  u'owner': u'PATRIC@patricbrc.org',
  u'patric_id': u'fig|266.10.peg.418',
  u'pgfam_id': u'PGF_00066260',
  u'plfam_id': u'PLF_265_00007981',
  u'product': u'Urocanate hydratase (EC 4.2.1.49)',
  u'property': [u'EC number', u'Pathway', u'Subsystem'],
  u'public': True,
  u'refseq_locus_tag': u'SAMN04244581_03492',
  u'segments': [u'32094..33752

In [83]:
# You can use the GPR ID to link reactions to their annotated genes in the feature file.
model.reactions[150].gene_reaction_rule

u'266.10.peg.366'

In [85]:
# It's a bit clumsy, but it works... 
for i in range(0,len(model.reactions)):
    if feature_list[i]['patric_id'] == u'fig|266.10.peg.366':
        print(i)
feature_list[i]['patric_id']

522


u'fig|266.10.peg.4145'

In [11]:
# This function allows us to identify which reactions were gapfilled to allow for 'growth'. 
gf_solutions = mackinac.get_patric_gapfill_solutions(genome_id)

1

In [21]:
# PATRIC has a built in ModelSEED that is more up-to-date than the ModelSEED website
# There is a built in gapfill step to ensure that the model can grow on complete media.
# Complete media refers to having all exchange reactions open. 
# When the model is generate using PATRIC it is given exchange reactions for all of the 
# transporters it has evidence of along with other important transporters such as water. 

# There were 100 reactions gapfilled to allow for 'growth' on complete media
# The rxn IDs are the keys in this list(dict).
len(gf_solutions[0]['reactions'].keys())

100

In [30]:
# Model Stats: this are well within the range I would expect based on the reported numbers.
print(len(model.metabolites))
print(len(model.reactions))
print(len(model.genes))
MODEL

1490
1434
966


Name,266.10
Memory address,0x07f0921b657d0
Number of metabolites,1490
Number of reactions,1434
Objective expression,-1.0*bio1_reverse_b18f7 + 1.0*bio1
Compartments,"Cytosol, Extracellular"


In [ ]:
# Here is the list of exchange reactions that are open, therefore the model has access to these
# metabolites for the next optimization step. 
model.medium

In [31]:
# Optimize using FBA: again very similar to the reported value!
opt = model.slim_optimize()
print('Objective value: %3.15f\n' % opt)

Objective value: 205.727409809327952



In [ ]:
# Save the model. This is considered a draft reconstruction.
cobra.io.save_json_model(model, "initMODEL.json")

In [58]:
# Here is a universal model. This model is a bit of a long story but it essentially contains all of 
# decent quality ModelSEED reactions and metabolites. Ideally you wouldn't have any imbalances when
# gapfilling with this reaction bag. 

univ_master = cobra.io.load_json_model("../Data/Near_complete_ModelSEED_universal.json")

In [66]:
univ_master

Name,MicrobialNegative
Memory address,0x07f091d72ead0
Number of metabolites,11392
Number of reactions,14709
Objective expression,0
Compartments,"c, e"


In [54]:
# You might consider removing the previous gapfilled reactions if you have some decent experimental
# data to work with. The best/cheapest data to work with is growth/no-growth data in defined media 
# conditions. You will want to use this to gapfill in the future. Here's an example: 

slim_model = model.copy() # It's best to deep.copy your model or universal before altering it. If 
# don't you will likely run into solver issues. Cobrapy has it's own deep.copy function. 

for rxn in gf_solutions[0]['reactions'].keys():
    slim_model.reactions.get_by_id(rxn).remove_from_model(remove_orphans=True)

In [55]:
slim_model

Name,266.10
Memory address,0x07f091cc252d0
Number of metabolites,1446
Number of reactions,1334
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"Cytosol, Extracellular"


In [60]:
# This is just a simple example media you could gapfill on. It has no biological significance. 
medium = [
    'cpd00067',
    'cpd00001',
    'cpd00011',
    'cpd00007',
    'cpd00528',
    'cpd00205',
    'cpd00971',
    'cpd00254',
    'cpd00030',
    'cpd10515', 
    'cpd00063',
    'cpd03422',
    'cpd00644',
    'cpd01415',
    'cpd00029',
    'cpd00137',
    'cpd00443',
    'cpd00013',
    'cpd00009',
    'cpd00048',
    'cpd00182',
    'cpd00307',
    'cpd00311',
    'cpd00184',
    'cpd00249',
    'cpd00027', 
    # Metals and ions
    'cpd00030',
    'cpd00034',
    'cpd00048', # Sulfate
    'cpd00058',
    'cpd00063',
    'cpd00099',
    'cpd00149',
    'cpd00205',
    'cpd00254',
    'cpd00971',
    'cpd10515',
    'cpd10516',
    'cpd00028',# Heme
    'cpd00268' # H2S2O3
]

In [63]:
sys.stdout.write('Load models...')

# NOTE THE NAME CHANGES HERE (Copying takes some time, most of the time it's worth it)
universal = univ_master.copy()
model = slim_model.copy()

sys.stdout.write('Set-up...')

# Ensure free diffusion of water (The ModelSEED water transporter is no good)
universal.reactions.get_by_id('rxn05319_c').name = "Water transport"
universal.reactions.get_by_id('rxn05319_c').bounds = (-1000., 1000.)
model.reactions.get_by_id('rxn05319_c').name = "Water transport"
model.reactions.get_by_id('rxn05319_c').bounds = (-1000., 1000.)

# Ensure the objective is biomass production ('growth')
# This biomass function is super generic, you will want to update this in the future. However,
# this is one of the more challanging/unknown aspects of modeling, in regards to it's importance and
# utility. It largely depends on your applications. 
model.objective = model.reactions.get_by_id('bio1')

Load models...Set-up...

In [64]:
model

Name,266.10
Memory address,0x07f091f305c90
Number of metabolites,1446
Number of reactions,1334
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"Cytosol, Extracellular"


In [65]:
universal

Name,MicrobialNegative
Memory address,0x07f091f305bd0
Number of metabolites,11392
Number of reactions,14709
Objective expression,0
Compartments,"c, e"


In [76]:
# pFBA-based gapfilling code that's way faster than the subpar Cobrapy gapfill function.
# In this gapfilling code you will see some more complex techniques I use to quickly gapfill a 
# model by minimizing the sum of fluxes through reactions from the universal while allowing 
# free flow (with no minimization) through the original model network. This will be far easier
# to explain during a skype call. 

# Establish from base values. 
lower_bound=0.05
penalties=None
demand_reactions=False
exchange_reactions=False
flux_cutoff=1E-10
exchange_prefix='EX_'

# Deep.copy the models
gapfiller = universal.copy()
model_to_gapfill = model.copy()

'''
General steps:
Add EXs to Universal (Create new objects)
Add Media metabolites to model if not present (Copy from Universal)
Add EXs to model (Copy from universal)
Add Transporters to Model (Copy from Universal)
'''

sys.stdout.write('Add EXs...')
# Add Exchanges to Gapfiller, just in case we have new-to-the-model metabolites in the media
for met in gapfiller.metabolites:
    base_met_id = met.id.split('_')[0]
    if 'EX_' + base_met_id + '_e' not in set([reaction.id for reaction in gapfiller.reactions])\
    and base_met_id + '_e' in set([metabolite.id for metabolite in gapfiller.metabolites]):
        # Create exchange reaction and add to model
        new_exchange = cobra.Reaction('EX_' + base_met_id + '_e')
        new_exchange.name = base_met_id + ' exchange'
        met_obj = gapfiller.metabolites.get_by_id(base_met_id + '_e')
        new_exchange.add_metabolites({met_obj:-1})
        new_exchange.lower_bound = -1000.
        new_exchange.upper_bound = 1000.
        gapfiller.add_reaction(new_exchange)
        gapfiller.repair()
    elif 'EX_' + base_met_id + '_e' in set([reaction.id for reaction in gapfiller.reactions]):
        gapfiller.reactions.get_by_id('EX_'+base_met_id+'_e').upper_bound = 1.0*1000

sys.stdout.write('Add Mets...')
# Add media metabolites to model_to_gapfill from gapfiller
for base_met_id in medium:
    if base_met_id + '_e' not in set([metabolite.id for metabolite in model_to_gapfill.metabolites])\
    and base_met_id + '_e' in set([metabolite.id for metabolite in gapfiller.metabolites]):
        # Add metabolite from gapfiller
        MET = gapfiller.metabolites.get_by_id(base_met_id + '_e').copy()
        model_to_gapfill.add_metabolites([MET])
        model_to_gapfill.repair()
    if base_met_id + '_c' not in set([metabolite.id for metabolite in model_to_gapfill.metabolites])\
    and base_met_id + '_c' in set([metabolite.id for metabolite in gapfiller.metabolites]):
        # Add metabolite from gapfiller
        MET = gapfiller.metabolites.get_by_id(base_met_id + '_c').copy()
        model_to_gapfill.add_metabolites([MET])
        model_to_gapfill.repair()

sys.stdout.write('Copy EXs...')
# Add missing Exchanges to model_to_gapfill from gapfiller
for met in model_to_gapfill.metabolites:
    base_met_id = met.id.split('_')[0]
    if 'EX_' + base_met_id + '_e' not in set([reaction.id for reaction in model_to_gapfill.reactions])\
    and 'EX_' + base_met_id + '_e' in set([reaction.id for reaction in gapfiller.reactions]):
        # Add Exchange from gapfiller
        EX = gapfiller.reactions.get_by_id('EX_' + base_met_id + '_e').copy()
        model_to_gapfill.add_reactions([EX])
        model_to_gapfill.repair()

sys.stdout.write('Transporters...')
# Identify transporters for each biolog component in the universal model
# and pick one that will enable transport in the gapfilling problem.
pick_transporter = {}
transporters_in_universal = [rxn for rxn in gapfiller.reactions if len(rxn.compartments)>1]
for base_met_id in medium:
    metabolite = model_to_gapfill.metabolites.get_by_id(base_met_id+'_c')
    rxns_with_metabolite = metabolite.reactions
    transport = False
    for rxn in rxns_with_metabolite:
        metabolites = [met_in_rxn.id for met_in_rxn in rxn.metabolites]
        if (base_met_id+'_e' in metabolites and base_met_id+'_c' in metabolites):
            transport = True

    if not transport:
        print("missing transporter for " + metabolite.name)
        for rxn in transporters_in_universal:
            metabolites = [met_in_rxn.id for met_in_rxn in rxn.metabolites]
            if (base_met_id+'_e' in metabolites and base_met_id+'_c' in metabolites):
                pick_transporter[base_met_id] = rxn.id

# Add the transporters to the model
transporters_to_add = list(pick_transporter.values())
transporter_list = []
for rxn in transporters_to_add:
    transporter_list.append(gapfiller.reactions.get_by_id(rxn).copy())
model_to_gapfill.add_reactions(transporter_list)

# Grab the linear reaction coefficients from the model 
original_objective = linear_reaction_coefficients(model_to_gapfill)
# convert to IDs to avoid issues with model membership when these reactions are added to gapfiller
original_objective = {rxn.id:original_objective[rxn] for rxn in original_objective.keys()}

# get the reactions in the original model, which need to be removed from
# the universal if present. This cannot catch identical reactions that do
# not share IDs, so make sure your model and universal are in the same
# namespace.
rxns_to_remove = [rxn for rxn in gapfiller.reactions if rxn.id in \
                    [rxn.id for rxn in model_to_gapfill.reactions]]
gapfiller.remove_reactions(rxns_to_remove)

# get the list of reactions currently in the gapfiller, which are the ones
# we will need to check for flux after solving the problem (e.g. these are
# the reactions we are considering adding to the model)
get_fluxes = [rxn.id for rxn in gapfiller.reactions]

# add the reactions from the model to the gapfiller, which are not
# included in the pFBA formulation, and thus flux is not penalized
# through them.
original_model_reactions = [rxn.copy() for rxn
                            in model_to_gapfill.reactions]
gapfiller.add_reactions(original_model_reactions)
original_reaction_ids = [reaction.id for reaction
                            in original_model_reactions]

# Add the pFBA constraints and objective (minimizes sum of fluxes)
add_pfba(gapfiller)

# set the linear coefficients for reactions in the original model to 0
coefficients = (gapfiller.objective
                .get_linear_coefficients(gapfiller.variables))
reaction_variables = (((gapfiller.reactions.get_by_id(reaction)
                        .forward_variable),
                       (gapfiller.reactions.get_by_id(reaction)
                        .reverse_variable))
                        for reaction in original_reaction_ids)
variables = chain(*reaction_variables)
for variable in variables:
    coefficients[variable] = 0.0 # might need to be 0.001
gapfiller.objective.set_linear_coefficients(coefficients)

# set a constraint on flux through the original objective
for reaction in original_objective.keys():
    print("Constraining lower bound for " + reaction)
    gapfiller.reactions.get_by_id(reaction).lower_bound = lower_bound

# Close import for all exchanges
exchange_reactions = [rxn for rxn in gapfiller.reactions if rxn.id.startswith(exchange_prefix)]
for rxn in exchange_reactions:
    rxn.lower_bound = 0

# Set the medium for this condition.
# set_media(gapfiller, medium, universal)
for ex_rxn in medium:
    gapfiller.reactions.get_by_id('EX_'+ex_rxn+'_e').lower_bound = -1.0*1000
    gapfiller.reactions.get_by_id('EX_'+ex_rxn+'_e').upper_bound = 1.0*1000

# gapfill and get the solution
iteration_solution = gapfiller.optimize()

filtered_solution = {rxn:iteration_solution.x_dict[rxn] for rxn in\
   get_fluxes if abs(iteration_solution.x_dict[rxn]) > flux_cutoff}

add_rxns = [gapfiller.reactions.get_by_id(rxn).copy() for \
                                rxn in filtered_solution.keys()]

def validate(original_model, reactions, lower_bound):
        with original_model as model:
            model.add_reactions(reactions)
            mets = [x.metabolites for x in reactions]
            all_keys = set().union(*(d.keys() for d in mets))
            model.add_metabolites(all_keys)
            model.slim_optimize()
            return (model.solver.status == OPTIMAL and
                    model.solver.objective.value >= lower_bound)

# validate that the proposed solution restores flux through the
# objective in the original model
# set the bounds on the original model to represent media
# and validate the gapfill solution
for ex_rxn in [rxn for rxn in model_to_gapfill.reactions if rxn.id.startswith(exchange_prefix)]:
    ex_rxn.lower_bound = 0

for ex_rxn in medium:
    model_to_gapfill.reactions.get_by_id('EX_'+ex_rxn+'_e').lower_bound = -1.0*1000
    model_to_gapfill.reactions.get_by_id('EX_'+ex_rxn+'_e').upper_bound = 1.0*1000
# set_media(model_to_gapfill, medium, universal)

print(len(add_rxns))

if not validate(model_to_gapfill, [gapfiller.reactions.get_by_id(rxn.id).copy() for rxn in add_rxns], lower_bound):
    raise RuntimeError('Failed to validate gapfilled model, '
                        'try lowering the flux_cutoff through '
                        'inclusion_threshold')

new_model = model_to_gapfill.copy()
reactions = [gapfiller.reactions.get_by_id(rxn.id).copy() for rxn in add_rxns]
new_model.add_reactions(reactions)
mets = [x.metabolites for x in reactions]
all_keys = set().union(*(d.keys() for d in mets))
new_model.add_metabolites(all_keys)
new_model.slim_optimize()

Add EXs...Add Mets...Copy EXs...Transporters...missing transporter for N2
missing transporter for Fe2+
missing transporter for Ca2+
missing transporter for Cobinamide
missing transporter for Thioglycolate
missing transporter for Acetate
missing transporter for ABEE
missing transporter for Adenosine
missing transporter for Guanosine
missing transporter for Thymidine
missing transporter for Uridine
missing transporter for D-Glucose
missing transporter for Ca2+
missing transporter for Co2+
missing transporter for Fe2+
missing transporter for Heme
Constraining lower bound for bio1
488


185.65780881952182

In [77]:
# This media condition is much more minimal than complete media which is why so many reactions 
# were added to the model. You'll notice if you re-run this a few times you'll get different solutions,
# this is to be expected. Additional growth conditions will help to narrow the solution space. 
new_model

Name,266.10
Memory address,0x07f0908b2da10
Number of metabolites,1898
Number of reactions,1855
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"Cytosol, Extracellular"


In [ ]:
# There are some other packages that help to better link genes/annotations to reactions, what is
# currently annoying about this field is that there are not many great tools out there for generating
# a solid link between the genes and reactions. Probannopy is the best place to start and I can help
# you work with that package as well! I'm currently developing some code that uses Probannopy reaction
# likelihoods to preferentially gapfill. The inputs for Probannopy are a fasta file, PATRIC genome ID, 
# and a unique template model. The downside is that the template model only contains 6500 reactions. 
# So it's limited by that template model, which I'm also trying to figure out how to expand to account
# for all of the reactions in the universal model file I sent you. I think your best bet would be to 
# use a variety of methods to automatically generate models with a strong connections to the annotations
# I'm happy to be your point person for questions and help. As I said before a lot of this is stuff 
# I'm building projects around currently, so it won't be a bother at all! 